# Traitement des données avec spark

## Importation des bibliothèques

In [22]:
import sys
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, explode, to_timestamp, hour, date_format
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

## Initialisation de Spark et Glue

In [23]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


## Définition des chemins S3

In [24]:
S3_raw_datas = "s3://raw-datas-projet/finance/"
S3_output = "s3://raw-datas-projet/output/"

## Lecture en streaming des nouvelles données

In [25]:
df_streaming = (
    spark.readStream
    .option("multiline", "true")
    .schema("results ARRAY<STRUCT<"  
            "id: STRING, name: STRING, symbol: STRING, current_price: DOUBLE, "
            "market_cap: LONG, total_volume: LONG, high_24h: DOUBLE, low_24h: DOUBLE, "
            "price_change_24h: DOUBLE, price_change_percentage_24h: DOUBLE, "
            "ath: DOUBLE, ath_date: STRING, atl: DOUBLE, atl_date: STRING, last_updated: STRING, "
            "roi: STRUCT<currency: STRING, percentage: DOUBLE, times: DOUBLE>>>")
    .json(S3_raw_datas)
)

## Transformation en streaming

In [26]:
df_exploded = df_streaming.withColumn("results", explode(df_streaming["results"]))

df_flattened = df_exploded.select(
    "results.id",
    "results.name",
    "results.symbol",
    "results.current_price",
    "results.market_cap",
    "results.total_volume",
    "results.high_24h",
    "results.low_24h",
    "results.price_change_24h",
    "results.price_change_percentage_24h",
    "results.ath",
    "results.ath_date",
    "results.atl",
    "results.atl_date",
    "results.last_updated",
    "results.roi.currency",
    "results.roi.percentage",
    "results.roi.times"
)


## Conversion des dates en TimestampType

In [14]:
date_columns = ["ath_date", "atl_date", "last_updated"]
for col_name in date_columns:
    df_flattened = df_flattened.withColumn(col_name, to_timestamp(df_flattened[col_name], "yyyy-MM-dd'T'HH:mm:ss.SSSX"))


## Extraction des heures et jours pour des analyses dashboard ou calcus d'indicateurs

In [15]:
df_flattened = df_flattened.withColumn("hour_updated", hour(df_flattened["last_updated"]))
df_flattened = df_flattened.withColumn("day_updated", date_format(df_flattened["last_updated"], "EEEE"))


## Filtrons les cryptos avec un volume de transaction significatif

In [16]:
df_flattened = df_flattened.filter(col("total_volume") > 1000)

## Calcul de la volatilité sur 24h 

In [17]:
from pyspark.sql.functions import expr

df_flattened = df_flattened.withColumn("volatility_24h", 
                                       expr("(high_24h - low_24h) / current_price * 100"))

## Écriture en streaming vers S3 

In [18]:
query_csv = (
    df_flattened
    .writeStream
    .format("csv") 
    .option("checkpointLocation", "s3://raw-datas-projet/checkpoints/")  
    .option("path", S3_output)  
    .option("header", "true")  
    .option("delimiter", ",")  
    .outputMode("append")  
    .start()
)



## Streaming

In [21]:
query_csv.awaitTermination()

StreamingQueryException: Query [id = d39d586d-b74e-4244-8786-cb64403029b3, runId = cdd4ea7b-74e9-4912-b487-5a2cfab0694b] terminated with exception: Multiple streaming queries are concurrently using s3://raw-datas-projet/checkpoints/sources/0
